In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [4]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [5]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [6]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [7]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [8]:
df = tripdata[selected_features]

In [9]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### 30% of missingness MCAR

In [10]:
## get the sample size such as 5%, 10%,20% etc
def getSampleSize(df, perc=1):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def induceMissingValues(X_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def induceMissingValuesForDrop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [11]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [12]:
fill_dict ={'start_lat': 40.79865054, 'start_lon': -74.2404433, 'end_lat': 40.79165072 , 'end_lon':-74.28593088,'usertype':'agent',
            'hour': 24, 'temp': 38.6, 'feelslike':49.4,'dew':-25.7,'snowdepth':5.10,'winddir':380.0,'sealevelpressure':819.3,
            'visibility':7.2,'solarradiation': 1109,'conditions': 'muddy','dist': 5.345,'birthyear': 2003,
            'holiday':'unknown','day': 'special','month': 'unknown','gender': 'nonbinary'
        }

In [13]:
def ampute_each_variables(variable, method, X_train, y_train, X_test, y_test):
    if method == "remove":
        X_train , y_train = induceMissingValuesForDrop(X_train,y_train,variable,perc=30)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = induceMissingValuesForDrop(X_test,y_test,variable,perc=30)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train  = induceMissingValues(X_train,variable,perc=30)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test  = induceMissingValues(X_test,variable,perc=30)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
    if method == "bad":
        print("Simulate Imputation for BAD")
        X_train.fillna(fill_dict[variable], inplace=True)
        print('Train  after imputation', X_train.shape, y_train.shape)
        X_test.fillna(fill_dict[variable], inplace=True)
        print('Test  after imputation', X_test.shape, y_test.shape)

        
            
    return X_train , y_train, X_test , y_test

In [14]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [15]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [16]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [14]:
bootstrap_data_drop ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())   
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  18926
X_test Amputed value count of varaible  18926
Simulate Imputation for NA
Train after NA value removal Imputation (176642, 21) (176642,)
Test after NA value removal Imputation (44161, 21) (44161,)
started bootstrap run is  2023-03-19 20:35:16.212099
stopped bootstrap run is  2023-03-19 20:36:23.160710
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (6308

Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  18926
X_test Amputed value count of varaible  18926
Simulate Imputation for NA
Train after NA value removal Imputation (176642, 21) (176642,)
Test after NA value removal Imputation (44161, 21) (44161,)
started bootstrap run is  2023-03-19 21:08:27.560840
stopped bootstrap run is  2023-03-19 21:09:29.902540
Now running model is .... solarradiation
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  18926
X_test Amputed value count of varaible  18926
Simulate Impu

In [15]:
bootstrap_data_drop

{'start_lat': {'R2': [0.5195951498594004, 0.5373766119070005],
  'RMSE': [2.206574562181675, 2.2604242709175177],
  'intercept': [-731.8993167119424, -326.8442481178196]},
 'start_lon': {'R2': [0.5195951498594004, 0.5373766119070005],
  'RMSE': [2.206574562181675, 2.2604242709175177],
  'intercept': [-731.8993167119424, -326.8442481178196]},
 'end_lat': {'R2': [0.5195951498594004, 0.5373766119070005],
  'RMSE': [2.206574562181675, 2.2604242709175177],
  'intercept': [-731.8993167119424, -326.8442481178196]},
 'end_lon': {'R2': [0.5195951498594004, 0.5373766119070005],
  'RMSE': [2.206574562181675, 2.2604242709175177],
  'intercept': [-731.8993167119424, -326.8442481178196]},
 'usertype': {'R2': [0.5195951498594004, 0.5373766119070005],
  'RMSE': [2.206574562181675, 2.2604242709175177],
  'intercept': [-731.8993167119424, -326.8442481178196]},
 'hour': {'R2': [0.5195951498594004, 0.5373766119070005],
  'RMSE': [2.206574562181675, 2.2604242709175177],
  'intercept': [-731.8993167119424, 

### Mode Replacement

In [16]:
bootstrap_data_mode ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 21:41:47.026877
stopped bootstrap run is  2023-03-19 21:43:04.109861
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (6308

started bootstrap run is  2023-03-19 22:21:34.296556
stopped bootstrap run is  2023-03-19 22:22:51.288384
Now running model is .... dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 22:24:05.879837
stopped bootstrap run is  2023-03-19 22:25:27.722766
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Im

In [17]:
bootstrap_data_mode

{'start_lat': {'R2': [0.5197175845939133, 0.5358102975696991],
  'RMSE': [2.2083419428859608, 2.255752753064212],
  'intercept': [-687.6893415553639, -384.8289394366618]},
 'start_lon': {'R2': [0.5190939806087669, 0.5352580470399915],
  'RMSE': [2.2094101464476763, 2.2571665584317513],
  'intercept': [-618.4077171196108, -281.4889855626135]},
 'end_lat': {'R2': [0.5197158210271821, 0.5357324418564879],
  'RMSE': [2.2083597951033065, 2.2558730459541456],
  'intercept': [-719.5651635527945, -409.32699094443933]},
 'end_lon': {'R2': [0.519469538090721, 0.5355090230418147],
  'RMSE': [2.2089336407439415, 2.2565658639171735],
  'intercept': [-806.134419654295, -477.85205929486483]},
 'usertype': {'R2': [0.5154491612540603, 0.5317322147116816],
  'RMSE': [2.2193041026016544, 2.2657016015656293],
  'intercept': [-662.9846338034063, -376.8432858447302]},
 'hour': {'R2': [0.518741702819662, 0.5347564947956308],
  'RMSE': [2.2104138033031435, 2.2575481474475936],
  'intercept': [-657.37136364009

### Mean Replacement

In [18]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

bootstrap_data_mean = {}
for var in variables_for_mean_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 22:47:30.154157
stopped bootstrap run is  2023-03-19 22:48:51.193974
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (6308

In [19]:
bootstrap_data_mean

{'start_lat': {'R2': [0.5197232796487041, 0.5358191969478783],
  'RMSE': [2.208335986748187, 2.255768196291484],
  'intercept': [-693.224388263072, -384.09190130084653]},
 'start_lon': {'R2': [0.5191014090282093, 0.5352592916426211],
  'RMSE': [2.209357188478804, 2.2571260114791114],
  'intercept': [-606.9838735645355, -266.40626809194447]},
 'end_lat': {'R2': [0.5197215872070337, 0.5357440751417517],
  'RMSE': [2.2083481475028286, 2.255829434561083],
  'intercept': [-732.6000048403387, -416.8875643801731]},
 'end_lon': {'R2': [0.5194830863480774, 0.5355306144054152],
  'RMSE': [2.2089111470328415, 2.2565843279204243],
  'intercept': [-814.4773731849042, -480.1916135758756]},
 'hour': {'R2': [0.5192573948488828, 0.5352634646454537],
  'RMSE': [2.20926138455476, 2.256362633557327],
  'intercept': [-653.5829286252381, -370.14159606594785]},
 'temp': {'R2': [0.519663828334133, 0.5357300156691294],
  'RMSE': [2.2084895404019127, 2.255790663148822],
  'intercept': [-644.2435143151085, -363.

### Sampling Replacement

In [20]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
started bootstrap run is  2023-03-19 23:24:17.403524
stopped bootstrap run is  2023-03-19 23:25:39.970083
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
started bootstrap run is  2023-03-19 23:26:55.037068
stopped bootstrap run is  2023-03-19 23:28:20.050943
Now running model is .... end_lat
Train (252346, 21) (252346,)
Test 

started bootstrap run is  2023-03-20 00:14:29.027453
stopped bootstrap run is  2023-03-20 00:15:49.034262
Now running model is .... day
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
started bootstrap run is  2023-03-20 00:19:01.685787
stopped bootstrap run is  2023-03-20 00:20:21.650286
Now running model is .... month
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
started bootstrap run is  2023-03-20 00:23:40.913818
stopped bootstrap run is 

In [21]:
bootstrap_data_sample

{'start_lat': {'R2': [0.5196879586110587, 0.5357804839748478],
  'RMSE': [2.2084092223956757, 2.2558058937343897],
  'intercept': [-656.81192426277, -350.80290265971286]},
 'start_lon': {'R2': [0.518876128049726, 0.5351027893095912],
  'RMSE': [2.2098450897272786, 2.25776198071612],
  'intercept': [-752.602225593413, -431.14510298516774]},
 'end_lat': {'R2': [0.5197019243146445, 0.5357360699510865],
  'RMSE': [2.208418028048653, 2.2558888880010723],
  'intercept': [-680.4051705921552, -361.79189001930035]},
 'end_lon': {'R2': [0.5192218194336478, 0.5353292637315985],
  'RMSE': [2.2094408361072677, 2.256995523251562],
  'intercept': [-667.1292028212176, -347.0454658339876]},
 'usertype': {'R2': [0.512435919858589, 0.528262388420881],
  'RMSE': [2.2271873956726873, 2.2733167247721013],
  'intercept': [-653.0712822681745, -369.53665455494934]},
 'hour': {'R2': [0.5188543486434894, 0.5348277427809698],
  'RMSE': [2.2103761474133146, 2.257749994751646],
  'intercept': [-658.6576799663112, -

### Sample Imputation

In [17]:
bootstrap_data_bad ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"bad",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_bad[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 15:50:55.024048
stopped bootstrap run is  2023-03-21 15:52:46.764666
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
st

stopped bootstrap run is  2023-03-21 17:03:01.630829
Now running model is .... dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 17:03:29.586060
stopped bootstrap run is  2023-03-21 17:07:10.064594
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,

In [18]:
bootstrap_data_bad

{'start_lat': {'R2': [0.5196159814029273, 0.5357447043971486],
  'RMSE': [2.208473058853637, 2.2559200559162167],
  'intercept': [-583.0838060488047, -300.8372708183234]},
 'start_lon': {'R2': [0.5186963757285634, 0.534918727796521],
  'RMSE': [2.210586799162642, 2.2583072448885515],
  'intercept': [-932.1392697347802, -650.2789944989088]},
 'end_lat': {'R2': [0.5195355804891141, 0.5356121611187418],
  'RMSE': [2.208763517988871, 2.2561944308311244],
  'intercept': [-538.1925594537815, -253.92457577115397]},
 'end_lon': {'R2': [0.518783313110196, 0.5349371344260052],
  'RMSE': [2.210260343499924, 2.2578635933061166],
  'intercept': [-421.9224006543228, -145.5775447158921]},
 'usertype': {'R2': [0.5155656843935188, 0.5317763665350337],
  'RMSE': [2.2189811724892534, 2.2654396961814935],
  'intercept': [-662.9540617515688, -376.9421942582104]},
 'hour': {'R2': [0.5182564567294838, 0.5342389379352634],
  'RMSE': [2.2117844895779166, 2.259215885113595],
  'intercept': [-661.4942089661057, 